# Import software libraries

In [1]:
# Import required libraries.
import sys           # Read system parameters.
import pandas as pd  # Manipulate and analyze data.
import sqlite3       # Manage SQL databases.

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- pandas {}'.format(pd.__version__))
print('- sqlite3 {}'.format(sqlite3.sqlite_version))

Libraries used in this project:
- Python 3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) 
[GCC 7.3.0]
- pandas 1.1.3
- sqlite3 3.30.1


# Examine the database

In [2]:
# Connect to SQLite database.
connection = sqlite3.connect('/home/jovyan/work/Project/data/prod_sample.db')

In [3]:
# List all the tables in the database.
cursor = connection.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table in tables:
    print(table[0])

stock_description
online_retail_history


# Read data from the `online_retail_history` table

In [4]:
# Write the query to be executed that selects everything from the online_retail_history table.
query_orh = 'SELECT * FROM online_retail_history'

# Use the read_sql function in pandas to read a query into a DataFrame.
df_orh = pd.read_sql(query_orh, connection)

# Preview the first five rows of the data.
df_orh.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


In [5]:
# Get the shape of the data.
df_orh.shape

(15321, 8)

# Read data from the `stock_description` table

In [6]:
# Write the query to be executed that selects everything from the stock_description table
query_sd = 'SELECT * FROM stock_description'

# Use the read_sql function in pandas to read a query into a DataFrame.
df_sd = pd.read_sql(query_sd, connection)

# Preview the first five rows of the data.
df_sd.head()

,StockCode,Description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE


In [7]:
# Get the shape of the data.
df_sd.shape

(3952, 2)

# Aggregate the `online_retail_history` and `stock_description` datasets

In [8]:
# Write a query to aggregate the two datasets so that you have the stock descriptions as well as the stock code.
query_join = '''SELECT online_retail_history.*, stock_description.Description
            FROM online_retail_history
            LEFT JOIN stock_description
            ON online_retail_history.StockCode = stock_description.StockCode'''

# Use the read_sql function in pandas to read a query into a DataFrame.
history_w_description = pd.read_sql(query_join, connection)

# Preview the first five rows of the data.
history_w_description.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


In [9]:
# Get the shape of the data.
history_w_description.shape

(17032, 9)

# Identify and fix corrupt or unusable data

In [10]:
# Check the value counts of the "Description" field.
history_w_description['Description'].value_counts()

CREAM HANGING HEART T-LIGHT HOLDER    2174
JUMBO BAG RED RETROSPOT               1960
?                                     1711
REGENCY CAKESTAND 3 TIER              1711
PARTY BUNTING                         1615
LUNCH BAG RED RETROSPOT               1421
ASSORTED COLOUR BIRD ORNAMENT         1405
POPCORN HOLDER                        1329
LUNCH BAG  BLACK SKULL.               1271
SET OF 3 CAKE TINS PANTRY DESIGN      1257
PACK OF 72 RETROSPOT CAKE CASES       1178
Name: Description, dtype: int64

In [11]:
# Remove rows where "Description" is just a question mark (?).
filtered_df = history_w_description[history_w_description['Description'] != '?']

# Preview the first five rows of the data.
filtered_df.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


# Identify and remove duplicates

In [12]:
# Identify all duplicated data.
duplicated_data = \
filtered_df[filtered_df.duplicated(keep = False)]

In [13]:
# Print the duplicated data.
duplicated_data.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
176,536863,20727,1,2010-12-03 11:19:00,1.65,u17967,United Kingdom,1.65,LUNCH BAG BLACK SKULL.
178,536863,20727,1,2010-12-03 11:19:00,1.65,u17967,United Kingdom,1.65,LUNCH BAG BLACK SKULL.
496,537781,84879,8,2010-12-08 12:46:00,1.69,u17341,United Kingdom,13.52,ASSORTED COLOUR BIRD ORNAMENT
497,537781,84879,8,2010-12-08 12:46:00,1.69,u17341,United Kingdom,13.52,ASSORTED COLOUR BIRD ORNAMENT
569,537955,20725,1,2010-12-09 11:28:00,1.65,u16782,United Kingdom,1.65,LUNCH BAG RED RETROSPOT


In [14]:
# Remove the duplicated data.
filtered_df_no_duplicates = \
filtered_df[~filtered_df.duplicated()]

# Preview the first five rows of the data.
filtered_df_no_duplicates.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


# Correct date formats

In [15]:
# Get the data types for every column in the DataFrame.
filtered_df_no_duplicates.dtypes

Invoice         object
StockCode       object
Quantity         int64
InvoiceDate     object
Price          float64
CustomerID      object
Country         object
TotalAmount    float64
Description     object
dtype: object

In [16]:
# Convert "InvoiceDate" to a "%Y-%m-%d" datetime format.
filtered_df_no_duplicates['InvoiceDate'] = pd.to_datetime(filtered_df_no_duplicates['InvoiceDate'], format='%Y-%m-%d')

In [17]:
# Get the data types for every column in the converted DataFrame.
filtered_df_no_duplicates.dtypes

Invoice                object
StockCode              object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
CustomerID             object
Country                object
TotalAmount           float64
Description            object
dtype: object

# Examine the table before finishing

In [18]:
# Preview the first five rows of the data.
filtered_df_no_duplicates.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


# Load the dataset into a pickle file

In [19]:
# Save the dataset as a pickle file named online_history_cleaned.pickle.
filtered_df_no_duplicates.to_pickle('online_history_cleaned.pickle')

In [20]:
# Close any connections to the database.
connection.close()